In [1]:
# default_exp performer

# Performer
> The base class for performing inference

In [6]:
#hide
from nbdev.showdoc import show_doc

In [14]:
#export
from fastreinference.configure import InferenceConfiguration, ManagerType
from fastcore.transform import Transform, Pipeline, _get_name

import torch
from fastai.learner import Learner

from contextlib import contextmanager

In [15]:
#export
def _print_fastai_transform(transform:Transform):
    "Prints information about a `transform`"
    information_string = "Transform Name: "
    information_string += f"`{transform.__module__}.{_get_name(transform)}`\n"
    information_string += f"Supported input types:\n"
    for dispatch in list(transform.encodes.funcs.d.keys()):
        information_string += f"  - `{dispatch.__module__}.{_get_name(dispatch)}`\n"
    information_string += "Has different behavior on test set:\n"
    information_string += "  - "
    if transform.split_idx is None:
        if hasattr(transform, "before_call"):
            information_string += "Yes"
        else:
            information_string += "No"
    elif transform.split_idx == 0:
        information_string += "Yes, is not performed"
    print(information_string)

In [5]:
#export
class Performer:
    _pipeline_events = ["type_transforms", "item_transforms", "batch_transforms"]
    _in_training_state = False
    def __init__(self, model, transform_pipelines:dict = None):
        """
        An encapsulating inference module.
        
        `transform_pipeline` should be a dictionary of general transform pipelines
          with keys: `type_transforms`, `item_transforms`, and `batch_transforms`
          
        Transforms do not need to be fastai transforms, but be cognizant that if mixing is performed,
          you should be wary of the fastai dispatch system and ensure that the pipeline can continue.
          
        Item and Batch transforms should either be all fastai transforms or all custom transforms.
        """
        self.model = model
        if transform_pipelines is None:
            print("""Warning: No `transform_pipelines` were passed to `Performer.__init__`. 
            Please set them with `Performer.configure_pipelines` before performing inference!""")
        else:
            missing_events = []
            for key in _pipeline_events:
                if key not in transform_pipelines.keys():
                    missing_events += key
            if len(missing_events) > 0:
                raise ValueError(f'Passed `pipes` dictionary is missing one or more keys: {missing_events}')
            self.type_transforms = transform_pipelines["type_transforms"]
            self.item_transforms = transform_pipelines["item_transforms"]
            self.batch_transforms = transform_pipelines["batch_transforms"]
            
    @classmethod
    def from_learner(cls, learner:Learner):
        """
        Builds a `Performer` from a fastai `Learner`
        """
        model = learner.model
        transform_pipelines = {}
        transform_pipelines["type_transforms"] = learner.dls.train.tfms
        transform_pipelines["item_transforms"] = learner.dls.after_item
        transform_pipelines["batch_transforms"] = learner.dls.after_batch
        cls(model, transform_pipelines)
        
        
    def _are_transforms_fastai(self, transforms:list, transform_kind:str):
        """
        Checks if all transforms in `transforms` are from fastai. Will raise an error if there is mismatching.
        """
        all_from_fastai = all(
            isinstance(transform, Transform) 
            for transform in transforms
        )
        any_from_fastai = any(
            isinstance(transform, Transform)
            for transform in transforms
        )
        
        if (any_from_fastai and not all_from_fastai) or (not all_from_fastai and any_from_fastai):
            raise ValueError(f'''All passed {transform_kind} are not of the same kind.
            Please use either fastai transforms or custom transforms.
            
            fastai Transforms:
              {[t for t in transforms if isinstance(transform, Transform)]}
            Custom Transforms:
              {[t for t in transforms if not isinstance(transform, Transform)]}''')
            
        if not any_from_fastai: 
            return False
        if all_from_fastai:
            return True
    
    # Should this be broken up given the single-responsibility principal?
    # Performer should only be getting batches and applying events
    # Need a new DL type or pipeline-type class
    def configure_pipelines(self, type_transforms:list, item_transforms:list, batch_transforms:list):
        """
        Override and configure all data preparation pipelines based on lists of transforms.
        Transforms do not need to be fastai transforms, but be cognizant that if mixing is performed,
          you should be wary of the fastai dispatch system and ensure that the pipeline can continue.
          
        Item and Batch transform lists should either be all fastai transforms or all custom transforms.
        """
        self.type_transforms = type_transforms
        if self._are_transforms_fastai(item_transforms, "`item_transforms`"):
            self.item_transforms = Pipeline(item_transforms)
        else:
            self.item_transforms = item_transforms
        if self._are_transforms_fastai(batch_transforms, "`batch_transforms`"):
            self.batch_transforms = Pipeline(batch_transforms)
        else:
            self.batch_transforms = batch_transforms
        print("Successfully configured transform pipelines")
        
    def describe_pipeline_of_type(self, pipeline_type:str):
        """
        Describes a configured transform pipeline.
        Can be a string of "type", "item", or "batch".
        
        Transforms are then described as:
        ```
        Transform Name: `Resize`
        Supported input types: 
          - `PIL.Image.Image`
          - `fastai.vision.core.TensorBBox`
          - `fastai.vision.core.TensorPoint`
        Has different behavior on test set: 
          - Yes
        ```
        if it is a fastai transform, else:
        ```
        Transform Name: `function_name`
        ```
        """
        if not any(pipeline_name == kind for kind in ["type", "item", "batch"]):
            raise ValueError(f"Invalid value passed for `pipeline_name`: {pipeline_name}. Must be one of 'type', 'item', or 'batch'.")
        transforms = getattr(self, f"{pipeline_name}_transforms")
        print('-'*36)
        if isinstance(transforms, Pipeline):
            for transform in transforms: 
                _print_fastai_transform(transform)
                print('-'*36)
        else:
            for transform in transforms:
                print(f"Transform Name: `{_get_name(transform)}`")
                print('-'*36)
                
    def _call_pipeline(self, inputs, pipeline_type:str):
        "Runs `self.{pipeline_type}` on inputs"
        pipeline = getattr(self, pipeline_type)
        if isinstance(pipeline, Pipeline):
            return pipeline(inputs)
        else:
            for transform in pipeline: 
                inputs = transform(inputs)
            return inputs
        
    @contextmanager
    def in_training_state(self):
        """
        Context manager that sets data preprocessing to apply training augmentation while in use
        """
        original_split_idxs = []
        for event in self._pipeline_events:
            pipe = getattr(self, event)
            if isinstance(pipe, Pipeline):
                original_split_idxs.append(pipe.split_idx)
                setattr(pipe, "split_idx", 0)
        yield
        for event in self._event_pipeline_events:
            pipe = getattr(self, event)
            if isinstance(pipe, Pipeline):
                setattr(pipe, "split_idx", original_split_idxs[0])
                original_split_idxs.pop(0)
    
    @staticmethod
    def _get_context_manager(manager_type:ManagerType):
        if manager_type == "no_grad": 
            return torch.no_grad
        elif manager_type == "inference": 
            return torch.inference_mode
        else:
            raise NotImplementedError(f"Context manager of type {manager_type} is not supported. Please use a valid `ManagerType`.")
        
    # def _do_inference(self, config):
        
                
    def predict(self, inputs, config, **kwargs):
        if isinstance(inputs, fastai.data.load.DataLoader) or isinstance(fastai.data.load.TfmdDL):
            dataloader = inputs
        elif isinstance(inputs, torch.utils.data.DataLoader):
            dataloader = inputs
        else:
            for event in self._pipeline_events:
                inputs = self._call_pipeline(inputs, event)
            dataloader = torch.utils.data.DataLoader(inputs, **kwargs)
            
        if config.context != ManagerType.NONE:
            with get_context_manager(config.context)():
                return self._do_inference(dataloader, config)
        else:
            return self._do_inference(dataloader, config)

AttributeError: module 'fastai' has no attribute 'learn'

In [27]:
show_doc(Performer.describe_pipeline_of_type)

<h4 id="Performer.describe_pipeline_of_type" class="doc_header"><code>Performer.describe_pipeline_of_type</code><a href="__main__.py#L65" class="source_link" style="float:right">[source]</a></h4>

> <code>Performer.describe_pipeline_of_type</code>(**`pipeline_type`**:`str`)

Describes a configured transform pipeline.
Can be a string of "type", "item", or "batch".

Transforms are then described as:
```
Transform Name: `Resize`
Supported input types: 
  - `PIL.Image.Image`
  - `fastai.vision.core.TensorBBox`
  - `fastai.vision.core.TensorPoint`
Has different behavior on test set: 
  - Yes
```
if it is a fastai transform, else:
```
Transform Name: `function_name`
```